In [1]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import Ollama
from string import Template
from sentence_transformers import SentenceTransformer
import numpy as np
import matplotlib.pyplot as plt

from base_components import select_action, get_outcomes, update_situation, mental_deliberation

# Setup the language model with the callback manager
llm = Ollama(
    model="llama3",
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)

# Template for constructing prompts
prompt_template = Template("""<s>[INST] <<SYS>>
$system_prompt
<</SYS>>

$request 
Answer as if you were the character you are playing. Be as concise as possible. 
Answer:[/INST]""")

system_message = ("This is an agent based model. "
    f"The goal of the LLM to to play the characters. "
    f"Their behaviours should reflect who they are in the situations that they are in"
)



In [6]:
names = ["Bob", "Alice", "Dorthy", "Charlie"]

personalities = {
    "Bob": "cautious, always thinking about the worst case scenario",
    "Alice": "optimistic, always looking at the best outcomes, not the most reliable",
    "Dorthy": "scarcastic, thinks its funny when people fight",
    "Charlie": "peaceful, always looking for a way to resolve conflicts without fighting"
}

situation = "Alice crashed Bob's car, and we wants her to pay for the damage."

memories = {"Bob": [], "Alice": [], "Dorthy": [], "Charlie": []}

for i in range(4):
    bob_memories = prompt_string = " ".join(memories["Bob"])
    alice_memories = prompt_string = " ".join(memories["Alice"])
    dorthy_memories = prompt_string = " ".join(memories["Dorthy"])
    charlie_memories = prompt_string = " ".join(memories["Charlie"])

    print("Bob's action!")
    action_bob = select_action(llm, "Bob", personalities["Bob"], bob_memories, situation)
    print()
    print()
    print("Alice's action!")
    action_alice = select_action(llm, "Alice", personalities["Alice"], alice_memories, situation)
    print()
    print()
    print("Dorthy's action!")
    action_dorthy = select_action(llm, "Dorthy", personalities["Dorthy"], dorthy_memories, situation)
    print()
    print()
    print("Charlies' action!")
    action_charlie = select_action(llm, "Charlie", personalities["Charlie"], charlie_memories, situation)
    print()
    print()

    action_dict = {"Bob": action_bob, "Alice": action_alice, "Dorthy": action_dorthy, "Charlie": action_charlie}

    print("Outcomes!")
    outcomes = get_outcomes(llm, action_dict, personalities, memories, situation)
    print()
    print("New situation!")
    situation = update_situation(llm, situation, outcomes)
    print()

    print("deliberation bob")
    deliberation_bob = mental_deliberation(llm, "Bob", personalities["Bob"], bob_memories, outcomes, situation)
    print()
    print()
    print("deliberation alice")
    deliberation_alice = mental_deliberation(llm, "Alice", personalities["Alice"], alice_memories, outcomes, situation)
    print()
    print()
    print("deliberation dorothy")
    deliberation_dorthy = mental_deliberation(llm, "Dorthy", personalities["Dorthy"], dorthy_memories, outcomes, situation)
    print()
    print()
    print("deliberation charlie")
    deliberation_charlie = mental_deliberation(llm, "Charlie", personalities["Charlie"], charlie_memories, outcomes, situation)
    print()
    print()


    print()
    print()

    outcome_string = prompt_string = " ".join(outcomes)

    memories["Bob"].append(f"Round {i+1}: " + outcome_string + deliberation_bob + "/n")
    memories["Alice"].append(f"Round {i+1}: " + outcome_string + deliberation_alice  + "/n")
    memories["Dorthy"].append(f"Round {i+1}: " + outcome_string + deliberation_dorthy + "/n")
    memories["Charlie"].append(f"Round {i+1}: " + outcome_string + deliberation_charlie  + "/n")




Bob's action!
[/INST]

Oh no, this can't be good... I'll think carefully before acting.

Actions:

1. Take pictures of damage for insurance.
2. Talk to Alice and try to calm down.
3. Call insurance company to report incident.
4. Confront Alice about responsibility.
5. Seek mediation from a third party.

I'll choose action 1: "Take pictures of damage for insurance."

[/INST]

Alice's action!
[/INST]

I'm Bob! Given Alice's careless mistake, I'll:

1. Stay calm & discuss solution with her.
2. Remind her of our friendship & offer help to fix it together.
3. Ask her to pay for the damage without making a scene.
4. Suggest we both take responsibility and get insurance quotes.
5. Politely tell her I need time to think about how to handle this.

[/INST]

I'll choose: "Ask her to pay for the damage without making a scene."

Dorthy's action!
What a delightful mess! I'll play Alice and see how this drama unfolds.

As {name}, a sarcastic agent, my options are:

1. Mock Bob's clumsiness.
2. Joke a

In [5]:
action_bob

"[Ugh, this is a nightmare!] As Bob, I'm thinking:\n\n1. Call Alice's insurance company to report the damage.\n2. Try to negotiate with Alice for her to pay out of pocket.\n3. Report the incident to my own insurance company (just in case).\n4. Offer to help Alice fix the car myself (to avoid a bigger mess).\n5. Just stay calm and try not to make things worse.\n\nI'll take... **2. Try to negotiate with Alice**."